In [1]:
pip install numpy matplotlib tensorflow logging

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import make_blobs
from matplotlib.widgets import Slider
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

In [5]:
def my_softmax(z):
    ez = np.exp(z)              #element-wise exponential
    sm = ez/np.sum(ez)
    return(sm)

# make  dataset for example
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
X_train, y_train = make_blobs(n_samples=2000, centers=centers, cluster_std=1.0,random_state=30)
print(y_train.shape)

In [10]:
model = Sequential(
    [ 
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'softmax')    # < softmax activation here. note 4 neuron used, not 1
    ]
)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    X_train,y_train,
    epochs=10
)
   

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 559us/step - loss: 0.7972 
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 512us/step - loss: 0.3583
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - loss: 0.1649
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 497us/step - loss: 0.0954
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 500us/step - loss: 0.0673
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step - loss: 0.0538
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 548us/step - loss: 0.0465
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 509us/step - loss: 0.0404
Epoch 9/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step - loss: 0.0363
Epoch 10/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 526us/step - loss: 0.0331


In [11]:
p_nonpreferred = model.predict(X_train)
print(p_nonpreferred [:2])
print("largest value", np.max(p_nonpreferred), "smallest value", np.min(p_nonpreferred))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 625us/step
[[1.0431834e-03 9.4714505e-04 9.8002177e-01 1.7987978e-02]
 [9.9777853e-01 1.9992229e-03 1.2271269e-04 9.9427722e-05]]
largest value 0.9999993 smallest value 5.289033e-11


In the preferred organization the final layer has a linear activation. For historical reasons, the outputs in this form are referred to as logits. The loss function has an additional argument: from_logits = True. This informs the loss function that the softmax operation should be included in the loss calculation. This allows for an optimized implementation.

In [12]:
preferred_model = Sequential(
    [ 
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'linear')   #<-- Note
    ]
)
preferred_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  #<-- Note
    optimizer=tf.keras.optimizers.Adam(0.001),
)

preferred_model.fit(
    X_train,y_train,
    epochs=10
)
        

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 523us/step - loss: 1.0242 
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 526us/step - loss: 0.4195
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step - loss: 0.1967
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 515us/step - loss: 0.1180
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - loss: 0.0864
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step - loss: 0.0703
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 539us/step - loss: 0.0607
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 542us/step - loss: 0.0542
Epoch 9/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 539us/step - loss: 0.0492
Epoch 10/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 567us/step - loss: 0.0455


In [13]:
p_preferred = preferred_model.predict(X_train)
print(f"two example output vectors:\n {p_preferred[:2]}")
print("largest value", np.max(p_preferred), "smallest value", np.min(p_preferred))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 607us/step
two example output vectors:
 [[-4.0424485  -4.0515575   2.8247514  -0.65841246]
 [ 3.0383065  -2.1797817  -4.2133145  -7.577049  ]]
largest value 8.402275 smallest value -13.5931


The output predictions above are not probabilities! If the desired output are probabilities, the output should be be processed by a softmax.

In [14]:
sm_preferred = tf.nn.softmax(p_preferred).numpy()
print(f"two example output vectors:\n {sm_preferred[:2]}")
print("largest value", np.max(sm_preferred), "smallest value", np.min(sm_preferred))

two example output vectors:
 [[1.0083324e-03 9.9918921e-04 9.6825725e-01 2.9735273e-02]
 [9.9388623e-01 5.3845546e-03 7.0468918e-04 2.4386325e-05]]
largest value 0.99999964 smallest value 4.0499049e-10


In [15]:
for i in range(5):
    print( f"{p_preferred[i]}, category: {np.argmax(p_preferred[i])}")

[-4.0424485  -4.0515575   2.8247514  -0.65841246], category: 2
[ 3.0383065 -2.1797817 -4.2133145 -7.577049 ], category: 0
[ 2.0595624 -1.2577902 -3.4014475 -6.0034075], category: 0
[-1.9841408  2.6365573 -2.5541856 -2.6526706], category: 1
[-1.8208609 -5.889757   3.7671826 -4.020607 ], category: 2


Tensorflow has two potential formats for target values and the selection of the loss defines which is expected.

SparseCategorialCrossentropy: expects the target to be an integer corresponding to the index. For example, if there are 10 potential target values, y would be between 0 and 9.
CategoricalCrossEntropy: Expects the target value of an example to be one-hot encoded where the value at the target index is 1 while the other N-1 entries are zero. An example with 10 potential target values, where the target is 2 would be [0,0,1,0,0,0,0,0,0,0].